In [ ]:
from bs4 import BeautifulSoup
import requests
import smtplib
import time
import datetime
import pandas as pd
import csv

In [ ]:
original_price = {}

In [ ]:
def get_details(URL):
    headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.63 Safari/537.36"} 
    page_html = requests.get(URL , headers=headers)
    page = BeautifulSoup(page_html.content, "html.parser")

    title = page.find("span", {"id": "productTitle"}).get_text()
    price = page.find("span", {"id": "priceblock_ourprice"}).get_text()
    rating = page.find(id='acrPopover').get_text()
    details = page.find(class_='a-normal a-spacing-micro').get_text()
    bank_offer = page.find(class_='description').get_text()
    img = page.find_all("img", class_ = "a-dynamic-image a-stretch-vertical")[0]["src"]

    title = title.strip()
    price= price.strip()
    price = price.replace('₹', '')
    price = price.replace(',', '')
    price = price.replace('.', '')
    price = price[:-2]
    rating = rating.strip()[:3]
    details = details.strip().replace('\n', '\t')
    bank_offer = bank_offer.strip()
    img = img.strip()

    print(title)
    print(price)
    print(rating)
    print(details)
    print(bank_offer)
    print(img)
    today = datetime.date.today()
    global original_price
    original_price[title] = int(price)
    
    return [title, price, today, rating, details, bank_offer, img]

In [ ]:
URL1 = 'https://www.amazon.in/JBL-Portable-Waterproof-Bluetooth-Champagne/dp/B081JZT5F6/?_encoding=UTF8&pd_rd_w=1NpvK&pf_rd_p=f162e864-1910-44b8-be2f-51a3819521a4&pf_rd_r=PRP996FKX3Q49F8ZHM38&pd_rd_r=8462549d-9cc1-43a7-9a37-427be48e60a0&pd_rd_wg=PGt7J&ref_=pd_gw_unk&th=1'

URL2 = 'https://www.amazon.in/Prisha-India-Craft-Copper-Bottle/dp/B06XJRDYMT/ref=sr_1_1?dchild=1&keywords=water+bottle&nav_sdd=aps&pd_rd_r=d7e32361-0b3c-4323-9f4a-36fbc22e0077&pd_rd_w=mEv1s&pd_rd_wg=Y62ge&pf_rd_p=39adf3af-15d6-4b91-9571-393f1f6517ef&pf_rd_r=QCHB828P4C6VJ5MKY6AH&qid=1631037777&refinements=p_36%3A3444813031&s=kitchen&sr=1-1'

URL3 = "https://www.amazon.in/Apple-MacBook-16-inch-Storage-2-3GHz/dp/B081JWZSSX/ref=sr_1_3?dchild=1&keywords=macbook+pro+16&qid=1631040132&sr=8-3"

URL4 = "https://www.amazon.in/Solimo-Three-Seater-Recliner-Brown/dp/B07FKKC81K/ref=sr_1_9?crid=15ZT74UG3V1ML&dchild=1&keywords=recliner+sofa&qid=1631040243&sprefix=recline%2Caps%2C318&sr=8-9"

URL5 = "https://www.amazon.in/Black-DrinkMate-Carbonated-Maker-Cylinder/dp/B018K1C3KK/ref=sr_1_3?dchild=1&keywords=soda+maker&qid=1631040304&sr=8-3"

speaker = get_details(URL1)
bottle = get_details(URL2)
macbook = get_details(URL3)
recliner = get_details(URL4)
sodamaker = get_details(URL5)

In [ ]:
headers = ['product', 'price', 'date', 'rating', 'details', 'bank_offer', 'img']

with open('Amazon_Web_Scraper_Project.csv', 'w', newline='', encoding='UTF8') as f:
  writer = csv.writer(f)
  writer.writerow(headers)
  writer.writerow(speaker)
  writer.writerow(bottle)
  writer.writerow(macbook)
  writer.writerow(recliner)
  writer.writerow(sodamaker)

In [ ]:
def send_mail(name, url):
    subject = f"The {name} you wanted so bad is now under the price you want."
    body = f"Dear Customer, this is the opportunity of a lifetime. Don't mess it up! Link here: {url}"
   
    msg = f"Subject: {subject}\n\n{body}"
    server = smtplib.SMTP_SSL('smtp.gmail.com',465)
    server.ehlo()
    #server.starttls()
    server.ehlo()
    server.login('isomerx2@gmail.com','fruits123')   
    server.sendmail('isomerx2@gmail.com', 'sanyajolly7@gmail.com', msg)

In [ ]:
def check_price(URL, multiplier):
  global original_price
  product = get_details(URL)


  if int(product[1]) <= original_price[product[0]]*multiplier:
    print(original_price)
    send_mail(product[0], URL)
  
  with open('Amazon_Web_Scraper_Project.csv', 'a+', newline='', encoding='UTF8') as f:
    writer = csv.writer(f)
    writer.writerow(product)

In [ ]:
while(True):
  check_price(URL1, 0.5)
  time.sleep(120)